```
Flores Aquino Luis Enrique
171564
```

# DEPENDENCIAS

In [1]:
#Instalacion
#Llamado
import numpy as np
import itertools as it

# DEFINICION DE APRIORI

## Metodos aledaños

Metodos que ayudan a la ejecución del algoritmo APRIORI

In [2]:
def GenerarPosibleItemset(Old_Itemset):
    PosItemSet = []
    for k in range(0,len(Old_Itemset)-1):
        for j in range(k+1,len(Old_Itemset)):
            #Realizamos la operacion A U B sobre 2 items antiguos
            Posible_Item = sorted(list(set(Old_Itemset[k]+Old_Itemset[j])))
            #Si el item resultante solo tiene 1 elemeno más que sus padres
            #Se le considera como posible itemset
            if(len(Posible_Item)==len(Old_Itemset[0])+1 
               and Posible_Item not in PosItemSet):
                PosItemSet.append(Posible_Item)
    return PosItemSet

def ContarAparicionItemSet(Pos_Itemset, Dataset):
    #Generamos un vector de igual longitud
    #Al posible item set para ir acumulando sus apareciones
    Contador_Tot = [0]*len(Pos_Itemset)
    for k in range(len(Pos_Itemset)):
        
        Item = Pos_Itemset[k]
        #Recorremos el dataset para contar las veces que contiene
            #el item actual
        for tupla in Dataset:
            #Hacemos la operación Item U Tupla
            tupla = set(tupla)
            temp = list(set(Item).union(tupla))
            tupla = list(tupla)
            #Luego comprobamos si cantidad de elementos
            #Si es igual a la de la tupla, en caso sea verdad
            #el item se encuentra dentro de la tupla
            if(len(temp)==len(tupla)):
                Contador_Tot[k]+=1
    return Contador_Tot


def LimpiarItemset(Itemset,Valores,umbral):
    #Funcion que se encarga de eliminar los valores del itemset
    #Cuya confianza no supere el umbral dado
    items_validos = []
    for valor_act in Valores:
        items_validos.append(valor_act>=umbral)
    Itemset_limpio = list(it.compress(Itemset,items_validos))
    Valores_limpio = list(it.compress(Valores,items_validos))
    return Itemset_limpio,Valores_limpio



#Obtenemos todas las permutaciones posibles
def ObtenerPosiblesReglas(item):
    combinaciones = list(it.permutations(item))
    Xs_actuales = []
    Ys_actuales = []
    for tupla in combinaciones:
        tupla = list(tupla)
        #print(tupla)
        #Obtenemos todas las divisiones posibles
        #de cada colección de items
        for k in range(1,len(tupla)):
            #Basandonos en que las reglas son
            # X -> Y
            # los separamos para en 2 variables
            X, Y = tupla[:k],tupla[k:]

            # Ordenamos cada sub conjunto
            X.sort() 
            Y.sort()

            #La agregamos al total
            Xs_actuales.append(X)
            Ys_actuales.append(Y)
    return Xs_actuales,Ys_actuales


def LimpiarReglasRepetidas(X,Y):
    #Eliminamos las combinaciones iguales 
    validos = [True]*len(X)
    for k in range(len(X)-1):
        #Si el valor actual todavía es válido
        #Comprobamos si tiene algún otro repetido
        if(validos[k]):
            #Comprobamos si entre los de adelante
            #hay una igual a la regla actual
            for j in range(k+1,len(X)):
                #Si el elemento j todavía es valido
                #con X y Y iguales, se marca como
                # NO VALIDO
                if(validos[j] and 
                    X[k] == X[j] and
                    Y[k] == Y[j]):
                    validos[j]=False
    #Eliminamos los no validos, tanto para X como Y
    X = list(it.compress(X,validos))
    Y = list(it.compress(Y,validos))
    return X,Y

def MostrarReglas(Xs,Ys):
    for k in range(len(Xs)):
        print(f'{Xs[k]} -> {Ys[k]}')

## Obtener la frecuencia del itemset
En base a un umbral mínimo

In [3]:
def get_frequent_itemsets(playlists, min_support=0.2):
    #Creamos 2 arreglos
        #Uno almacenará todos los itemsets
        #El otro su soporte
    All_Itemsets = []
    All_values = []
    num_iter=1
    #Ordenamos las playlists
    #for playlist in playlists:
    #    playlist.sort()
    
    print('Calculando itemsets de long %2i'%(num_iter))
    #PRINT DE MUESTRA
    #for playlist in playlists:
    #    print(playlist)
    #print("??????")
    #Obtenemos todos los itemsets de 1
    itemset = []
    val_item = []
    for miniset in playlists:
        for item in miniset:
            item = str(item)
            if(item in itemset):
                val_item[itemset.index(item)]+=1
            else:
                itemset.append(item)
                val_item.append(1)
                
    #PRINT DE MUESTRA
    #for k in range(len(itemset)):
    #    print(itemset[k] + '|' + str(val_item[k]))

    #Normalizamos en valores de 0 a 1 los valores acumulados que se tienen
    long_playlists = len(playlists)
    val_item= [x/long_playlists for x in val_item]
    
    
    #Eliminamos todos los objetos que no pasan el umbra "min_support"
    valid_itemset,valid_values= LimpiarItemset(itemset,val_item,min_support)

    #Ordenamos los valores
    temp_duplas = dict(zip(valid_itemset,valid_values)).items()
    valid_itemset = []
    valid_values = []
    for dupla in sorted(temp_duplas):
        valid_itemset.append(dupla[0])
        valid_values.append(dupla[1])
    
    #PRINT DE MUESTRA
    #for k in range(len(valid_itemset)):
    #   print(valid_itemset[k] + '|' + str(valid_values[k]))

    #PROCEDEMOS A INICIAR UN CICLO PARA CALCULAR ITEMSETS DE MAYOR CARDINALIDAD
    last_itemset = [[x] for x in valid_itemset]

    #PRINT DE MUESTRA
    print("└──Terminado || cant. elementos (%i)"%(len(last_itemset)))
    
    while(True):
        
        #Comprobamos si nuestra última iteración tiene todavía items válidos
        if(len(last_itemset)==0):
            break

        #Añadimos nuestra última iteracion al total de itemsets
        All_Itemsets.append(last_itemset)
        All_values.append(valid_values)

        #Si solo tenemos un item en el itemset tambien acabamos todo
        if(len(last_itemset)==1):
            break
        
        num_iter+=1
        print('Calculando itemsets de long %2i'%(num_iter))

        #Creamos todas las tuplas posibles de la próxima iteración
        Pos_Itemset = GenerarPosibleItemset(last_itemset)
        #Contamos las apariciones del posible item set a lo largo
            #de todo el data set
        Val_PosItems = ContarAparicionItemSet(Pos_Itemset,playlists)
                  
        #Normalizamos en valores de 0 a 1 los valores acumulados que se tienen

        Val_PosItems= [x/long_playlists for x in Val_PosItems]

        #Eliminamos los valores que no pasan el umbral
        last_itemset, valid_values = LimpiarItemset(Pos_Itemset,Val_PosItems,min_support)
        #PRINT DE MUESTRA
        #for k in range(len(Pos_Itemset)):
        #    print(str(Pos_Itemset[k])+'|'+ str(Val_PosItems[k]))
        print("└──Terminado || cant. elementos (%i)"%(len(last_itemset)))
    return All_Itemsets,All_values

## Generar las reglas de asociacion

In [4]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift =0):
    All_rules = []
    All_lift = []
    All_confidence = []
    All_support = []
    Long_regla_actual = 2
    itemsets = frequent_itemsets[0]
    itemsvalues = frequent_itemsets[1]
    Xs = []
    Ys = []
    print('Calculas todas las posibles reglas')
    while(Long_regla_actual<=len(itemsets)):
        #Obtenemos todas las posibles reglas
        itemset_actual = itemsets[Long_regla_actual-1]
        #Obtenemos las posibles regla de cada item
        for item in itemset_actual:
            xs,ys = ObtenerPosiblesReglas(item)
            Xs+=xs
            Ys+=ys
        Long_regla_actual+=1
    print('Limpieza de reglas repetidas')
    #Una vez con todas las posibles reglas
    #De la forma X->Y
    #Limpiamos las reglas repetidas
    Xs,Ys = LimpiarReglasRepetidas(Xs,Ys)
    

    #CALCULO DEL LIFT Y CONFIANZA
    #Calculo del lift
    
    #Recorremos las reglas y calculamos su lift
    #y confianza, además de verificar que pasen
    #Los umbrales
    print('Calculando lift y confianza')
    for k in range(len(Xs)):
        x = Xs[k]
        y = Ys[k]
        xuy = x+y
        xuy.sort()
        #Obtenemos los valores necesarios para el calculo
        pos = itemsets[len(x)-1].index(x)
        Px = itemsvalues[len(x)-1][pos]
        pos = itemsets[len(y)-1].index(y)
        Py = itemsvalues[len(y)-1][pos]
        pos = itemsets[len(xuy)-1].index(xuy)
        Pxuy = itemsvalues[len(xuy)-1][pos]
        #Calculamos la confianza y el lift
        confianza_act = Pxuy/Px
        lift_act = Pxuy/Px/Py
        #comprobamos si pasa el umbral
        if(lift_act >= lift and confianza_act >= confidence):
            All_rules.append([x,y])
            All_confidence.append(confianza_act)
            All_lift.append(lift_act)
            All_support.append(Pxuy)
    print('FIN')
    return  All_rules,All_confidence,All_lift,All_support

# EJECUCION

## LECTURA DE DATOS

In [5]:
data_spotify = np.load('spotify.npy',allow_pickle=True)

## TRATAMIENTO

In [6]:
X = data_spotify.tolist()
spotify_playlists = [X[k] for k in X]

# Ejecucion

## Obtencion de los itemset

In [7]:
All_Itemsets,All_values = get_frequent_itemsets(spotify_playlists, 0.02)
#Completo 12m 33.4s || 0.04
#         29m  1.6s || 0.02
#5000      4m 27.8s || 0.02

Calculando itemsets de long  1
└──Terminado || cant. elementos (161)
Calculando itemsets de long  2
└──Terminado || cant. elementos (3)
Calculando itemsets de long  3
└──Terminado || cant. elementos (0)


### Mostrar itemsets adquiridos

In [8]:
print(f'Longitud máxima encontrada: {len(All_Itemsets)}')
# MOSTRAR ITEMSETS VALIDOS ENCONTRADOS
for k in range(len(All_Itemsets)):
    print('===== %3i ======='%(k+1))
    for j in range(len(All_Itemsets[k])):
        print('%s || %3.2f'%(All_Itemsets[k][j],All_values[k][j]*100))

Longitud máxima encontrada: 2
=====   1 =======
['1-800-273-8255'] || 2.64
['679 (feat. Remy Boyz)'] || 2.46
['Again'] || 2.14
["Ain't No Mountain High Enough"] || 2.33
['Alive'] || 2.33
['All I Want'] || 2.34
['All of Me'] || 2.41
['Alone'] || 2.15
['Alright'] || 2.81
['Angel'] || 2.25
['Antidote'] || 2.64
['Bad and Boujee (feat. Lil Uzi Vert)'] || 3.53
['Bank Account'] || 2.31
['Beautiful'] || 2.02
['Believe'] || 2.20
['Believer'] || 2.54
['Black Beatles'] || 2.96
['Blessings'] || 2.84
['Body Like A Back Road'] || 2.73
['Bohemian Rhapsody - Remastered 2011'] || 2.43
['Bounce Back'] || 3.00
['Broccoli (feat. Lil Yachty)'] || 4.02
['CAN\'T STOP THE FEELING! (Original Song from DreamWorks Animation\'s "TROLLS")'] || 2.09
["Can't Feel My Face"] || 2.26
['Caroline'] || 3.95
['Cheap Thrills'] || 2.14
['Chicken Fried'] || 2.18
['Chill Bill'] || 2.30
['Closer'] || 7.61
['Cold Water (feat. Justin Bieber & MØ)'] || 2.41
['Come On Eileen'] || 2.07
['Congratulations'] || 4.10
['Controlla'] || 2.

## Calcular reglas de asociacion

In [9]:
Reglas,Confianza,Lift,Soporte = generate_association_rules([All_Itemsets,All_values],0.4,0.75)

Calculas todas las posibles reglas
Limpieza de reglas repetidas
Calculando lift y confianza
FIN


### Mostrar reglas de asociacion

In [10]:
for k in range(len(Reglas)):
    print(f"{k+1} ======")
    print(f'{Reglas[k][0]} -> {Reglas[k][1]}')
    print(f'soporte: {Soporte[k]*100}')
    print(f'confianza: {Confianza[k]*100}')
    print(f'lift: {Lift[k]}')

1 ======
['Congratulations'] -> ['HUMBLE.']
soporte: 2.1399999999999997
confianza: 52.195121951219505
lift: 11.10534509600415
2 ======
['HUMBLE.'] -> ['Congratulations']
soporte: 2.1399999999999997
confianza: 45.53191489361702
lift: 11.10534509600415
3 ======
['HUMBLE.'] -> ['Mask Off']
soporte: 2.04
confianza: 43.40425531914894
lift: 13.735523835173714
4 ======
['Mask Off'] -> ['HUMBLE.']
soporte: 2.04
confianza: 64.55696202531645
lift: 13.735523835173714
5 ======
['HUMBLE.'] -> ['XO TOUR Llif3']
soporte: 2.04
confianza: 43.40425531914894
lift: 13.152804642166345
6 ======
['XO TOUR Llif3'] -> ['HUMBLE.']
soporte: 2.04
confianza: 61.81818181818181
lift: 13.152804642166345


# INTENTO CON LA LIBRERÍA

In [12]:
from apyori import apriori
lon_min = 2 # Número mínimo de productos a considerar en las reglas

soporte_min = 0.02

confianza_min = 0.04
sustento_min = 0.75 #lift min
association_rules = apriori(spotify_playlists, min_support=soporte_min, min_confidence=confianza_min, min_lift=sustento_min, min_length=2)
association_results = list(association_rules)  #Convertir los resultados a lista para facilidad de visualización

# MOSTRAR DATA

In [13]:
# Visualizar todos los resultados
contador=1
for item in association_results:

    pair = item[0] 
    items = [x for x in pair]
    print(contador)
    print(f"Regla: {items}")
    print("Soporte: " + str(item[1]*100))
    print("Confianza: " + str(item[2][0][2]*100))
    print("Sustento | Lift: " + str(item[2][0][3]))
    print("=====================================")
    contador=contador+1

1
Regla: ['Closer']
Soporte: 7.23
Confianza: 7.23
Sustento | Lift: 1.0
2
Regla: ['Congratulations']
Soporte: 4.03
Confianza: 4.03
Sustento | Lift: 1.0
3
Regla: ['HUMBLE.']
Soporte: 4.65
Confianza: 4.65
Sustento | Lift: 1.0
4
Regla: ['Home']
Soporte: 4.54
Confianza: 4.54
Sustento | Lift: 1.0
5
Regla: ['One Dance']
Soporte: 4.109999999999999
Confianza: 4.109999999999999
Sustento | Lift: 1.0
6
Regla: ['Ride']
Soporte: 4.14
Confianza: 4.14
Sustento | Lift: 1.0
7
Regla: ['Roses']
Soporte: 4.24
Confianza: 4.24
Sustento | Lift: 1.0
8
Regla: ['Congratulations', 'HUMBLE.']
Soporte: 2.1399999999999997
Confianza: 53.10173697270471
Sustento | Lift: 11.419728381226818
9
Regla: ['HUMBLE.', 'Mask Off']
Soporte: 2.04
Confianza: 43.87096774193549
Sustento | Lift: 13.883217639853001
10
Regla: ['HUMBLE.', 'XO TOUR Llif3']
Soporte: 2.04
Confianza: 43.87096774193549
Sustento | Lift: 13.582342954159591
